# CREATE FULL WEATHER AND EVENTS DF

In [12]:
import requests
import pandas as pd
import dateutil.parser

## 1. Getting daily sunrise and sunset times from the Open Meteo API

### 1.1 Getting the data from the API

In [13]:
start_date = "2023-08-20"
end_date = "2023-08-23"
url = 'https://archive-api.open-meteo.com/v1/archive'

In [14]:
params_daily_dict ={
        "latitude":"51.5085", #London latitude - should remain hardcoded
        "longitude":"-0.1780971", #London longitude -should remain hardcoded
        "start_date":start_date, #could be defined in .env and used in the other files
        "end_date":end_date, #could be defined in .env and used in the other files
        "timezone":"Europe/London", #Europe/London - specific to this api
        "daily":"sunrise,sunset" # specific to this api
}

daily_weather_response = requests.get(
        url,
        params=params_daily_dict).json()

In [15]:
sun_df =pd.DataFrame()
sun_df["sunrise"] = daily_weather_response["daily"]["sunrise"]
sun_df["sunset"] = daily_weather_response["daily"]["sunset"]
sun_df.head()

,sunrise,sunset
0,2023-08-20T05:54,2023-08-20T20:14
1,2023-08-21T05:55,2023-08-21T20:12
2,2023-08-22T05:57,2023-08-22T20:10
3,2023-08-23T05:58,2023-08-23T20:07


### 1.2 Daytime/nightime encoding

In [16]:
def date_parser(time):
    date_parser = dateutil.parser.isoparse(time)
    return date_parser

In [17]:
sun_df["sunrise_datetime"] = sun_df.apply(lambda x: date_parser(x["sunrise"]), axis = 1)

In [18]:
sun_df["sunset_datetime"] = sun_df.apply(lambda x: date_parser(x["sunset"]), axis = 1)

In [19]:
sun_df['date'] = sun_df['sunrise_datetime'].dt.date

In [20]:
sun_df.head()

,sunrise,sunset,sunrise_datetime,sunset_datetime,date
0,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,2023-08-20
1,2023-08-21T05:55,2023-08-21T20:12,2023-08-21 05:55:00,2023-08-21 20:12:00,2023-08-21
2,2023-08-22T05:57,2023-08-22T20:10,2023-08-22 05:57:00,2023-08-22 20:10:00,2023-08-22
3,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00,2023-08-23


In [21]:
def daytime_encoding(timestamp, sunrise_datetime, sunset_datetime):
    if timestamp < sunrise_datetime:
        daytime_encoding = "nighttime"
    elif timestamp >= sunrise_datetime and timestamp < sunset_datetime:
        daytime_encoding = "daytime"
    else:
        daytime_encoding = "nighttime"
    return daytime_encoding

## 2. Getting hourly weather data from the Open Meteo API

### 2.1 API data

In [22]:
params_hourly_dict ={
        "latitude":"51.5085", #London latitude - should remain hardcoded
        "longitude":"-0.1780971", #London longitude -should remain hardcoded
        "start_date":start_date, #could be defined in .env and used in the other files
        "end_date":end_date, #could be defined in .env and used in the other files
        "timezone":"Europe/London", #Europe/London - specific to this api
        "hourly":"temperature_2m,precipitation,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m" # specific to this api
}

hourly_weather_response = requests.get(
    url,
    params=params_hourly_dict).json()

In [23]:
timestamp_api = hourly_weather_response["hourly"]["time"]
temperature_api = hourly_weather_response["hourly"]["temperature_2m"]
precipitation_api = hourly_weather_response["hourly"]["precipitation"]
rain_api = hourly_weather_response["hourly"]["rain"]
snow_api = hourly_weather_response["hourly"]["snowfall"]
cloudcover_api = hourly_weather_response["hourly"]["cloudcover"]
windspeed_api = hourly_weather_response["hourly"]["windspeed_10m"]
winddirection_api = hourly_weather_response["hourly"]["winddirection_10m"]

### 2.2 Put the data into a df

In [24]:
weather_data = pd.DataFrame()
weather_data["timestamp"] = timestamp_api
weather_data["temperature"] = temperature_api
weather_data["precipitation"] = precipitation_api
weather_data["rainfall"] = rain_api
weather_data["snowfall"] = snow_api
weather_data["cloudcover"] = cloudcover_api
weather_data["wind_speed"] = windspeed_api
weather_data["wind_direction"]= winddirection_api

In [25]:
weather_data

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction
0,2023-08-20T00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0
1,2023-08-20T01:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0
2,2023-08-20T02:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0
3,2023-08-20T03:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0
4,2023-08-20T04:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0
...,...,...,...,...,...,...,...,...
91,2023-08-23T19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2023-08-23T20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,2023-08-23T21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2023-08-23T22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.3 Timestamp recoding

In [26]:
weather_data["timestamp"] = weather_data.apply(lambda x: date_parser(x["timestamp"]), axis = 1)

In [27]:
weather_data.head()

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction
0,2023-08-20 00:00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0
1,2023-08-20 01:00:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0
2,2023-08-20 02:00:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0
3,2023-08-20 03:00:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0
4,2023-08-20 04:00:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0


### 2.4 Add the daytime & night time encoding to the full df

In [28]:
weather_data['date'] = weather_data['timestamp'].dt.date

In [29]:
weather_data

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction,date
0,2023-08-20 00:00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0,2023-08-20
1,2023-08-20 01:00:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0,2023-08-20
2,2023-08-20 02:00:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0,2023-08-20
3,2023-08-20 03:00:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0,2023-08-20
4,2023-08-20 04:00:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0,2023-08-20
...,...,...,...,...,...,...,...,...,...
91,2023-08-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23
92,2023-08-23 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23
93,2023-08-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23
94,2023-08-23 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23


In [30]:
weather_data = weather_data.merge(sun_df)

In [31]:
weather_data

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction,date,sunrise,sunset,sunrise_datetime,sunset_datetime
0,2023-08-20 00:00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
1,2023-08-20 01:00:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
2,2023-08-20 02:00:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
3,2023-08-20 03:00:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
4,2023-08-20 04:00:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2023-08-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00
92,2023-08-23 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00
93,2023-08-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00
94,2023-08-23 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00


In [32]:
weather_data["encoding"] = weather_data.apply(lambda x: daytime_encoding(x["timestamp"], x["sunrise_datetime"], x["sunset_datetime"]), axis = 1)

In [34]:
weather_data.head(20)

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction,date,sunrise,sunset,sunrise_datetime,sunset_datetime,encoding
0,2023-08-20 00:00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
1,2023-08-20 01:00:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
2,2023-08-20 02:00:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
3,2023-08-20 03:00:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
4,2023-08-20 04:00:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
5,2023-08-20 05:00:00,14.1,0.0,0.0,0.0,64.0,10.2,212.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
6,2023-08-20 06:00:00,14.0,0.0,0.0,0.0,88.0,10.2,212.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,daytime
7,2023-08-20 07:00:00,14.9,0.0,0.0,0.0,88.0,12.6,217.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,daytime
8,2023-08-20 08:00:00,16.5,0.0,0.0,0.0,94.0,15.3,223.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,daytime
9,2023-08-20 09:00:00,17.8,0.0,0.0,0.0,91.0,14.8,223.0,2023-08-20,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00,daytime


In [39]:
weather_data_final = weather_data.drop(columns={"date", "sunrise", "sunset"}, axis = 1)

In [40]:
weather_data_final

,timestamp,temperature,precipitation,rainfall,snowfall,cloudcover,wind_speed,wind_direction,sunrise_datetime,sunset_datetime,encoding
0,2023-08-20 00:00:00,15.9,0.0,0.0,0.0,5.0,10.6,208.0,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
1,2023-08-20 01:00:00,15.4,0.0,0.0,0.0,4.0,10.9,214.0,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
2,2023-08-20 02:00:00,15.0,0.0,0.0,0.0,5.0,10.4,214.0,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
3,2023-08-20 03:00:00,14.7,0.0,0.0,0.0,4.0,10.7,213.0,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
4,2023-08-20 04:00:00,14.4,0.0,0.0,0.0,27.0,10.5,211.0,2023-08-20 05:54:00,2023-08-20 20:14:00,nighttime
...,...,...,...,...,...,...,...,...,...,...,...
91,2023-08-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23 05:58:00,2023-08-23 20:07:00,daytime
92,2023-08-23 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23 05:58:00,2023-08-23 20:07:00,daytime
93,2023-08-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23 05:58:00,2023-08-23 20:07:00,nighttime
94,2023-08-23 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-23 05:58:00,2023-08-23 20:07:00,nighttime


## 3. Add events to main dataframe

### 3.1 Load the events_df

In [50]:
events_df = pd.read_csv("../raw_data/London_Events_vcsv2.csv")
print(events_df)

                   title  start_date    end_date Location     Latitude  \
0    Chelsea Flower Show  20/05/2014  24/05/2014  Chelsea  51.48692516   
1    Chelsea Flower Show  19/05/2015  23/05/2014  Chelsea  51.48692516   
2    Chelsea Flower Show  24/05/2016  28/05/2016  Chelsea  51.48692516   
3    Chelsea Flower Show  23/05/2017  27/05/2017  Chelsea  51.48692516   
4    Chelsea Flower Show  22/05/2018  26/05/2018  Chelsea  51.48692516   
..                   ...         ...         ...      ...          ...   
185                  NaN         NaN         NaN      NaN          NaN   
186                  NaN         NaN         NaN      NaN          NaN   
187                  NaN         NaN         NaN      NaN          NaN   
188                  NaN         NaN         NaN      NaN          NaN   
189                  NaN         NaN         NaN      NaN          NaN   

        Longitude  
0    -0.169618232  
1    -0.169618232  
2    -0.169618232  
3    -0.169618232  
4    -0.169

In [53]:
new_events_columns = ("event_title", "event_start_date", "event_end_date", "event_location", "event_lattitude", "event_longitude")

In [56]:
events_df.columns = new_events_columns

### 3.2 Add a date column to prepare for merge with main df

In [ ]:
events_df["date"] = weather_data['date'] = weather_data['timestamp'].dt.date

In [58]:
new_weat_tmp= weather_data_final.copy()

In [52]:
new_weat_tmp = new_weat_tmp.merge(events_df)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False